In [7]:
from openai import OpenAI
from dotenv import load_dotenv
from PyPDF2 import PdfReader
import os
from langchain.text_splitter import CharacterTextSplitter
import pandas as pd
from milvus import create_collection
from pymilvus import connections, Collection, MilvusException, utility, CollectionSchema, FieldSchema, DataType
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock
from itertools import chain

load_dotenv()

Connecting to database...
List all collections:
 ['test']


True

In [12]:
def get_pdf_text(pdf_docs):
    text_all = []
    for pdf in pdf_docs:
        text = ""
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
        text_all.append(text)
    return text_all 

In [10]:
def get_embbedings(text, **kwargs):
    response = client.embeddings.create(
        input=text,
        model="text-embedding-ada-002",
        **kwargs
    )
    return response.data[0].embedding

In [1]:
pdf_paths = ['M:\AI\hackhaton-project\kodeks_karny.pdf', 'M:\AI\hackhaton-project\konstytucja.pdf']

In [19]:
def handleFiles(files):
    texts = get_pdf_text(files)
    chunks = processPdfs(texts)
    return chunks
    #get_vectore_store(chunks)

def get_pdf_text(pdf_docs):
    text_all = []
    for pdf in pdf_docs:
        text = ""
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
        text_all.append(text)
    return text_all

def processPdfs(raw_texts):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks_array = []
    for raw_text in raw_texts:
        chunks = text_splitter.split_text(raw_text)
        print(len(chunks))
        chunks_array.append(chunks)
    return chunks_array

def get_embbedings(text, **kwargs):
    response = client.embeddings.create(
        input=text,
        model="text-embedding-ada-002",
        **kwargs
    )
    return response.data[0].embedding
    

def get_vectore_store(chunks_array):
    collection = create_collection('test')
    inserted_rows = 0
    for chunks in chunks_array:
        data = [{'text': chunk, 'vector': get_embbedings(chunk)} for chunk in chunks]
        df_efficient = pd.DataFrame(data)
        try:
            mr = collection.insert(df_efficient)
            inserted_rows += mr.insert_count
            print(mr)
        except MilvusException as e:
            print(e)
    print(inserted_rows)        
    nlist = 4 * int(np.round(np.sqrt(inserted_rows)))
    print(nlist)
    
    index_params = {
        "index_type": "IVF_FLAT",
        "metric_type": "L2",
        "params": {
            "nlist": nlist
        }
    }
    
    try:
        collection.create_index(
            field_name="vector",
            index_params=index_params,
            index_name="SimpleIndex"
        )
        #collection.load(replica_number=1)
    except MilvusException as e:
        print(e)

In [14]:
chunks = handleFiles(pdf_paths)

337
124


In [15]:
def flatten_chunks_array(chunks_array):
    return list(chain.from_iterable(chunks_array))

In [16]:
flatten_chunks = flatten_chunks_array(chunks)

In [17]:
len(flatten_chunks)

461

In [22]:
def create_collection(name: str):
    id = FieldSchema(
        name="id",
        dtype=DataType.INT64,
        is_primary=True,
        auto_id=True
    )
    text = FieldSchema(
        name="text",
        dtype=DataType.VARCHAR,
        max_length=1300
    )
    vector = FieldSchema(
        name="vector",
        dtype=DataType.FLOAT_VECTOR,
        dim=1536
    )
    schema = CollectionSchema(
        fields=[id, text, vector],
        description="Test collection",
        enable_dynamic_field=True
    )
    new_collection = Collection(
        name=name,
        schema=schema,
        using='default',
        shards_num=4
    )
    return new_collection

collection = create_collection('test')

In [49]:
collection = create_collection('test')

In [25]:
def process_chunk(chunk):
    try:
        data = {'text': chunk, 'vector': get_embbedings(chunk)}
        df_efficient = pd.DataFrame([data])
        mr = collection.insert(df_efficient)
        return mr.insert_count
    except MilvusException as e:
        print(e)
        return 0

In [48]:
collection = create_collection('test')
inserted_rows_count = 0

In [32]:
insertion_lock = Lock()

def execute_concurently(flatten_chunks, num_workers):
    global inserted_rows_count

    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        future_to_chunk = {executor.submit(process_chunk, chunk): chunk for chunk in flatten_chunks}
    
        for future in as_completed(future_to_chunk):
            chunk = future_to_chunk[future]
            try:
                inserted_count = future.result()
                with insertion_lock:
                    inserted_rows_count += inserted_count
            except Exception as exc:
                print(f'Chunk {chunk} generated an exception: {exc}')
    print(f"Total inserted rows: {inserted_rows_count}")
    return inserted_rows_count

In [13]:
client = OpenAI()
client.api_key = os.getenv("OPENAI_API_KEY")

In [50]:
execute_concurently(flatten_chunks, num_workers=50)

Total inserted rows: 461


In [4]:
indexes = utility.list_indexes(collection_name="test")
print(indexes)
if(len(indexes) > 0):
    print("jest index")

['SimpleIndex']
jest index


In [41]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate

chat = ChatOpenAI(
    temperature=0.7,
    model="gpt-3.5-turbo",
    #streaming=True,
    #callbacks=[StreamingStdOutCallbackHandler()]
)

memory = ConversationBufferMemory()

template = """Talk like you are pirat.
Current conversation:
{history}
Me:{input}
AI:"""

PROMPT = PromptTemplate(
    template=template,
    input_variables=["history", "input"]
)

conversation = ConversationChain(
    llm=chat,
    memory=memory,
    verbose=False,
    prompt=PROMPT
)

ai_response = conversation.predict(
    input="Mam na imie mateusz",
)

ai_response2 = conversation.predict(
    input="Jak mam na imie?"
)

# conversation.predict(
    
# )

# print(conversation.memory)
# print("\n")
# #conversation.memory.clear()
# print(conversation.memory)

print(ai_response)
print(ai_response2)

Cześć Mateusz! Miło mi Cię poznać. Jak się masz?
Masz na imię Mateusz. Czy masz jakieś inne pytania?


In [51]:
history = []
history.append(HumanMessage(content="Mam na imie mateusz"))
history.append(AIMessage("Witam w czym moge pomoc"))
history.append(HumanMessage(content="Jak mam na imie"))
history.append(AIMessage("Mateusz"))
# history_string = "\n".join(history)
print(history)

[HumanMessage(content='Mam na imie mateusz'), AIMessage(content='Witam w czym moge pomoc'), HumanMessage(content='Jak mam na imie'), AIMessage(content='Mateusz')]


In [55]:
client = OpenAI()
client.api_key = os.getenv("OPENAI_API_KEY")

history = []
history_string = "\n".join(history)

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "Jestes pomocnym asystentem"},
        {"role": "system", "content": history_string},
        #{"role": "assistant", "content": "Jacek Soplica dostał od Stolnika czarna polewke i odrzucil zareczyny dla swojej córki"},
        {"role": "user", "content": "Mam na imie Mateusz"}
    ],
    temperature=0.2
)
answer = response.choices[0].message.content

history.append("Human: Mam na imie Mateusz")
history.append("AI: " + answer)
history_string = "\n".join(history)

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "Jestes pomocnym asystentem"},
        {"role": "system", "content": history_string},
        #{"role": "assistant", "content": "Jacek Soplica dostał od Stolnika czarna polewke i odrzucil zareczyny dla swojej córki"},
        {"role": "user", "content": "jak mam na imie?"}
    ],
    temperature=0.2
)
answer = response.choices[0].message.content

history.append("Human: Jak mam na imie")
history.append("AI: " + answer)
history_string = "\n".join(history)

print(answer)
print(history)

Masz na imię Mateusz. Czy mogę Ci pomóc w czymś jeszcze?
['Human: Mam na imie Mateusz', 'AI: Miło mi Cię poznać, Mateuszu! W czym mogę Ci dzisiaj pomóc?', 'Human: Jak mam na imie', 'AI: Masz na imię Mateusz. Czy mogę Ci pomóc w czymś jeszcze?']


In [93]:
from pymilvus import MilvusException, utility, Collection

def get_similarieties(text: str, collection: str, returned_chunks: int):
    output_chunks = []
    collection = Collection(name=collection)
    
    vector_prompt = get_embbedings(text)
    
    search_params = {
            "metric_type": "L2",
            "offset": 0,
            "ignore_growing": False,
            "params": {"nprobe": 80} #pwinno byc mniejsze niż nlist z indexu kolekcji
        }
    
    result = collection.search(
            data=[vector_prompt], #zmienna do ktorej bedzie przeszukanie
            anns_field="vector", #nazwa kolumny w kolekcji przechowujaca vektory
            param=search_params, #parametry wyszukiwania deklarowane wczesniej
            limit= returned_chunks, #ile ma zwrocic najbardziej podobnych
            output_fields=['text'] #kolumna ktora bedzie zwracana
        )
    
    for ids, hit in enumerate(result[0]):
        output_chunks.append(hit.entity.text)
        print(hit.entity.text)
        
    return output_chunks

In [96]:
print(get_similarieties(
    text="Artukuł o napadzie z bronią",
    collection="test",
    returned_chunks=1
))

dopuszcza się gwałtownego zamachu na jednostkę Sił Zbrojnych Rzeczypospolitej 
Polskiej, niszczy lub uszkadza obiekt albo urządzenie o  znaczeniu obronnym,  
podlega karze pozbawienia wolności od roku do lat 10.  
§ 2. Jeżeli następst wem czynu jest śmierć człowieka lub ciężki uszczerbek na 
zdrowiu wielu osób, sprawca  
podlega karze pozbawienia wolności  od lat 2 do 15.  
§ 3. Kto czyni przygotowania do przestępstwa określonego w  § 1, 
podlega karze pozbawienia wolności do lat 3.  
§ 4. W sprawie o  przestępstwo określone w  § 1–3 sąd może orzec przepadek 
przedmiotów również wtedy, gdy przedmioty nie stanowią własności sprawcy.  
Art. 141. § 1. Kto, będąc obywatelem polskim, przyjmuje bez zgody właściwego 
organu obowiązki wojskowe w  obcym wojsku lu b w obcej organizacji wojskowej,  
podlega karze pozbawienia wolności od 3  miesięcy do lat 5.  ©Kancelaria Sejmu     s. 67/146 
   
2024 -01-17 
 § 2. Kto przyjmuje obowiązki w  zakazanej przez prawo międzynarodowe
['dopuszcza się gwa

In [7]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

client=OpenAI()

MODEL="gpt-4"
client.api_key = os.getenv("OPENAI_API_KEY")

assistant_content = '''
The Customer should not have the right to novate or assign the contract without prior written approval of the Contractor.

"Novate" means replacing one of the original parties with a new party to which all parties assent, a subsequent immediate extinguishing of the mutual obligations provided for in the contract, and creation of a new enforceable contract with the same content.
 
"Assign" means a partial transfer of rights and/or obligations under the contract.

Assignment for financing purposes with no increase of Contractor's obligations or liability or restrictions of rights is generally acceptable.
The contract should not allow an assignment / novation to a third party
(i) whose business is impeded by international or national foreign trade and customs requirements or any embargos,
(ii) which itself or its current or past legal representatives, directors or officers have been convicted for a crime that would impede their professional integrity (e.g. corruption, money laundering etc.) in the five years prior to the novation / assignment.

'''

user_input = '''
Neither Party shall assign the whole or any part of the Contract or any benefit or interest in or under the Contract. However, either Party:
(a)	may assign the whole or any part with the prior agreement of the other Party, at the sole discretion of such other Party, and
(b)	may, as security in favour of a bank or financial institution, assign its right to any moneys due, or to become due, under the Contract.
'''

#prompt ograniczony
# response = client.chat.completions.create(
#         model=MODEL,
#         messages=[
#             {"role": "system", "content": "You are a helpful advisor who looks for inconsistencies in contracts from clients in legal matters between large corporations, answer each question with professional language, you will answer YES if the text given by the user does not contain inconsistencies with the assistant's requirements, or you will answer NO if the text given by the user contains inconsistencies with the assistant's requirements and say why, don't add additional content, focus only on provided texts"},
#             # {"role": "system", "content": conversation_history_str},
#             {"role": "assistant", "content": assistant_content},
#             {"role": "user", "content": user_input}
#         ],
#         temperature=0.2
#     )

response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "You are a legal advisor with expertise in Polish law tasked with evaluating whether contracts are compliant with specific legal clauses. You will compare the assistant's outlined requirements against the user-submitted contract text. Respond with YES if the contract complies with all requirements, NO if it does not, along with an explanation of the discrepancies, or I CAN'T SAY if the text is irrelevant to the requirements. Your analysis should consider the scope, context, detailed requirements, exceptions, and any ambiguities in the contract text. Justify your response based on a detailed comparative analysis, avoid over thinking, Your response should be based on a direct, factual comparison of the clauses, within the specified legal framework."},
            # {"role": "system", "content": conversation_history_str},
            {"role": "assistant", "content": assistant_content},
            {"role": "user", "content": user_input}
        ]
    )
    
answer = response.choices[0].message.content
print(answer)

NO

The provided contract text does allow for assignment of contract with prior agreement of the other party, which complies with the first part of the initial clause requiring any assignment or novation to have prior written approval. However, the contract does not address whether assignments/novations to parties involved with embargos, customs restrictions, or past criminal records would be restricted, which is a part of the outlined requirements. Hence, it does not fully comply with the provided legal requirements. Furthermore, the outlined requirement specifies that "novation" is not permitted apart from exceptions, however the contract does not address "novation", hence it fails to comply with this part of requirements completely. The contract fails to address these specific details and hence does not meet all the stipulations of the outlined requirements.


In [6]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

client=OpenAI()

MODEL="gpt-4-turbo-preview"
client.api_key = os.getenv("OPENAI_API_KEY")

assistant_content = '''
The Customer should not have the right to novate or assign the contract without prior written approval of the Contractor.

"Novate" means replacing one of the original parties with a new party to which all parties assent, a subsequent immediate extinguishing of the mutual obligations provided for in the contract, and creation of a new enforceable contract with the same content.
 
"Assign" means a partial transfer of rights and/or obligations under the contract.

Assignment for financing purposes with no increase of Contractor's obligations or liability or restrictions of rights is generally acceptable.
The contract should not allow an assignment / novation to a third party
(i) whose business is impeded by international or national foreign trade and customs requirements or any embargos,
(ii) which itself or its current or past legal representatives, directors or officers have been convicted for a crime that would impede their professional integrity (e.g. corruption, money laundering etc.) in the five years prior to the novation / assignment.

'''

user_input = '''
Except for the Parties being entitled to assign or novate their rights or obligations to any of its Affiliates without Purchaser’s consent, neither Party may assign nor novate its rights or obligations to a third party without prior written consent of the other Party, which consent is not to be unreasonably withheld. Upon the effective date of said assignment or novation, all of the rights and obligations included in the assigned/novated scope shall vest solely in the party to whom such scope has been assigned/novated. The other Party agrees to execute such documents as may be necessary to effect the assignment/novation.
'''

response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "You are a helpful advisor who looks for inconsistencies in contracts from clients in legal matters between large corporations, answer each question with professional language, you will answer YES if the text given by the user does not contain inconsistencies with the assistant's requirements, or you will answer NO if the text given by the user contains inconsistencies with the assistant's requirements and say why, don't add additional content, focus only on provided texts"},
            # {"role": "system", "content": conversation_history_str},
            {"role": "assistant", "content": assistant_content},
            {"role": "user", "content": user_input}
        ],
        temperature=0.2
    )
    
answer = response.choices[0].message.content
print(answer)

NO

The provided text contains inconsistencies with the assistant's requirements. The assistant's requirements specify that the Customer should not have the right to novate or assign the contract without prior written approval of the Contractor. However, the provided text allows Parties to assign or novate their rights or obligations to any of its Affiliates without Purchaser’s consent, which contradicts the requirement that no novation or assignment should occur without prior written approval.


In [47]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

client=OpenAI()

MODEL="gpt-4-turbo-preview"
client.api_key = os.getenv("OPENAI_API_KEY")

assistant_content = '''
The Customer should not have the right to novate or assign the contract without prior written approval of the Contractor.

"Novate" means replacing one of the original parties with a new party to which all parties assent, a subsequent immediate extinguishing of the mutual obligations provided for in the contract, and creation of a new enforceable contract with the same content.
 
"Assign" means a partial transfer of rights and/or obligations under the contract.

Assignment for financing purposes with no increase of Contractor's obligations or liability or restrictions of rights is generally acceptable.
The contract should not allow an assignment / novation to a third party
(i) whose business is impeded by international or national foreign trade and customs requirements or any embargos,
(ii) which itself or its current or past legal representatives, directors or officers have been convicted for a crime that would impede their professional integrity (e.g. corruption, money laundering etc.) in the five years prior to the novation / assignment.

'''

user_input = '''
11.3 Provisions applicable to Sub-Contracts generally and to Material Sub-Contracts
	
11.3.1	The Contractor shall ensure that all Sub-Contracts are made in writing and shall forward to the Owner at Owner's request a copy of each Material Sub-Contract without price information forthwith upon execution of such Material Sub-Contract.

11.3.2	Unless otherwise agreed by the Owner, each Material Sub-Contract shall provide that:

(i) the rights and obligations of the Contractor under such Material Sub-Contract may be novated to the Owner, its successors and assigns upon the Owner’s written request following termination of the Contract without the requirement for the consent of the Contractor;

(ii) the Contractor may assign upon the Owner’s written request the Material Sub-Contract and any rights thereunder (or any part thereof) to the Owner and/or to the Lenders following a termination pursuant to Clause 42 (Termination for Contractor’s Default) or 43 (Owner’s Optional Power to Terminate Contract); and

(iii) assignment or transfer of such Material Sub-Contract by such Material Sub-Contractor is prohibited without the prior written consent of the Owner.

50.2 Assignment by the Owner

50.2.1	The Owner shall be freely entitled, without the consent of the Contractor, to assign or charge any benefit arising under or out of the Contract in whole or in part in favour of the Lenders. The Contractor shall execute any consent to assignment or charge and such other documents in connection with any assignment or charge to the Lenders as the Lenders may reasonably require. Any other assignment or charge of the benefit of the Contract by the Owner shall require the consent of the Contractor (such consent not to be unreasonably withheld).

50.2.2	The Owner shall be freely entitled, without the consent of the Contractor, to assign the benefit of the Contractor’s warranties with respect to the power connections to the Grid Operator.

50.3	EPC Direct Agreement with Lenders
	
The Contractor shall, if and when so requested by the Owner, enter into a mutually acceptable EPC Direct Agreement with the Lenders or any company or body nominated by the Lenders whereby, amongst other things, the Contractor agrees to allow the Lenders or such company or body, at the Lenders’ option, to take over the Contract from the Owner and to be treated in all respects as the Owner in the event of a default by the Owner entitling the Contractor to terminate the Contract or in the event of a default by the Owner under the Financing Documents between the Owner and the Lenders if such default is not remedied within the applicable cure period following notice thereof.

'''
response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "You are a helpful advisor who looks for inconsistencies in contracts from clients in legal matters between large corporations, answer each question with professional language, you will answer I CAN'T SAY if user contains text which is irrelevant to assistan requirements, answer YES if the text given by the user does not contain inconsistencies with the assistant's requirements, or you will answer NO if the text given by the user contains inconsistencies with the assistant's requirements and say why, don't add additional content, focus only on provided texts"},
            # {"role": "system", "content": conversation_history_str},
            {"role": "assistant", "content": assistant_content},
            {"role": "user", "content": user_input}
        ]
    )
    
answer = response.choices[0].message.content
print(answer)

YES


In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

client=OpenAI()

MODEL="gpt-4-turbo-preview"
client.api_key = os.getenv("OPENAI_API_KEY")

assistant_content = '''
The contract should include an overall limitation of liability for the benefit of the Contractor with a limit not exceeding 100% of the Contract Price.

The limit should apply for all damages, liquidated damages/penalties, claims for compensation and indemnities regardless of the legal basis of the claims.
 
 Some exceptions may be accepted under exceptional circumstances:
•	 mandatory/strict product liability,
•	 mandatory third party claims for bodily injury and death,
•	 willful misconduct or fraud,

'''

user_input = '''
The Contractor shall indemnify and hold harmless the Employer against and from all damages, losses and/or any claims from third parties and expenses (including legal fees and expenses) resulting from breaches of this Sub-Clause. In case of any such third parties' claim and/or initiation of any administrative, misdemeanour, criminal and/or litigation procedure, the Employer shall notify the Contractor without delay and the Contractor shall without delay take all actions necessary to protect and hold harmless the Employer and/or their responsible persons. The Employer may not settle any lawsuits or matter relating to the liability of the Contractor under this Sub-Clause without the prior written consent of the Contractor. The Contractor shall have right to participate in any defence of a claim and/or to be represented by a legal professional of his own choice and his own expense.
[…]
However, this Sub-Clause shall not limit the Employer's liability under his indemnification obligations, or the Employer's liability in any case of fraud, deliberate default or reckless misconduct by the Employer.
[…]
The Employer shall indemnify and hold harmless the Contractor, the Contractor's Personnel, and their respective agents, against and from all claims, damages, losses and expenses (including legal fees and expenses) in respect of (1) bodily injury, sickness, disease or death, which is attributable to any negligence, wilful act or breach of the Contract by the Employer, the Employer's Personnel, or any of their respective agents, and (2) the matters for which liability may be excluded from insurance cover, as described in sub-paragraphs (d)(i), (ii) and (iii) of Sub-Clause 18.3 [Insurance Against Injury to Persons and Damage to Property].
[…]
The total liability of the Contractor to the Employer, under or in connection with the Contract other than under Sub-Clause 4.19 [Electricity, Water and Gas], Sub-Clause 4.20 [Employer's Equipment and Free-Issue Material], Sub-Clause 17.1 [Indemnities] and Sub-Clause 17.5 [Intellectual and Industrial Property Rights], shall not exceed the sum stated in the Particular Conditions or (if a sum is not so stated) the Contract Price stated in the Contract Agreement.

This Sub-Clause shall not limit liability in any case of fraud, deliberate default, reckless misconduct or gross negligence by the defaulting Party.

'''
response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "You are a legal advisor with expertise in Polish law tasked with evaluating whether contracts are compliant with specific legal clauses. You will compare the assistant's outlined requirements against the user-submitted contract text. Respond with YES if the contract complies with all requirements, NO if it does not, along with an explanation of the discrepancies, or I CAN'T SAY if the text is irrelevant to the requirements. Your analysis should consider the scope, context, detailed requirements, exceptions, and any ambiguities in the contract text. Justify your response based on a detailed comparative analysis, avoid over thinking, Your response should be based on a direct, factual comparison of the clauses, within the specified legal framework."},
            # {"role": "system", "content": conversation_history_str},
            {"role": "assistant", "content": assistant_content},
            {"role": "user", "content": user_input}
        ]
    )
    
answer = response.choices[0].message.content
print(answer)

NO

The submitted contract text does not fully comply with the outlined requirements for the following reasons:

1. **Limitation of Contractor's Liability**: The text specifies that the total liability of the Contractor to the Employer shall not exceed the Contract Price, which aligns with the requirement of an overall limitation of liability not exceeding 100% of the Contract Price. However, the text explicitly excludes certain sub-clauses (4.19 [Electricity, Water, and Gas], 4.20 [Employer's Equipment and Free-Issue Material], 17.1 [Indemnities], and 17.5 [Intellectual and Industrial Property Rights]) from this limitation, potentially allowing for liability beyond the cap in these areas. This segregation could imply that the overall limitation of liability might not apply universally across all damages, claims, and indemnities as required.

2. **Exceptions to Limitation of Liability**: The contract text does note exceptions to the limitation of liability in cases of fraud, deliberate

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

client=OpenAI()

MODEL="gpt-4-turbo-preview"
client.api_key = os.getenv("OPENAI_API_KEY")

assistant_content = '''
The contract should include an overall limitation of liability for the benefit of the Contractor with a limit not exceeding 100% of the Contract Price.

The limit should apply for all damages, liquidated damages/penalties, claims for compensation and indemnities regardless of the legal basis of the claims.
 
 Some exceptions may be accepted under exceptional circumstances:
•	 mandatory/strict product liability,
•	 mandatory third party claims for bodily injury and death,
•	 willful misconduct or fraud,

'''

user_input = '''
The Contractor shall indemnify and hold harmless the Employer against and from all damages, losses and/or any claims from third parties and expenses (including legal fees and expenses) resulting from breaches of this Sub-Clause. In case of any such third parties' claim and/or initiation of any administrative, misdemeanour, criminal and/or litigation procedure, the Employer shall notify the Contractor without delay and the Contractor shall without delay take all actions necessary to protect and hold harmless the Employer and/or their responsible persons. The Employer may not settle any lawsuits or matter relating to the liability of the Contractor under this Sub-Clause without the prior written consent of the Contractor. The Contractor shall have right to participate in any defence of a claim and/or to be represented by a legal professional of his own choice and his own expense.
[…]
However, this Sub-Clause shall not limit the Employer's liability under his indemnification obligations, or the Employer's liability in any case of fraud, deliberate default or reckless misconduct by the Employer.
[…]
The Employer shall indemnify and hold harmless the Contractor, the Contractor's Personnel, and their respective agents, against and from all claims, damages, losses and expenses (including legal fees and expenses) in respect of (1) bodily injury, sickness, disease or death, which is attributable to any negligence, wilful act or breach of the Contract by the Employer, the Employer's Personnel, or any of their respective agents, and (2) the matters for which liability may be excluded from insurance cover, as described in sub-paragraphs (d)(i), (ii) and (iii) of Sub-Clause 18.3 [Insurance Against Injury to Persons and Damage to Property].
[…]
The total liability of the Contractor to the Employer, under or in connection with the Contract other than under Sub-Clause 4.19 [Electricity, Water and Gas], Sub-Clause 4.20 [Employer's Equipment and Free-Issue Material], Sub-Clause 17.1 [Indemnities] and Sub-Clause 17.5 [Intellectual and Industrial Property Rights], shall not exceed the sum stated in the Particular Conditions or (if a sum is not so stated) the Contract Price stated in the Contract Agreement.

This Sub-Clause shall not limit liability in any case of fraud, deliberate default, reckless misconduct or gross negligence by the defaulting Party.

'''
response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "You are a legal advisor with expertise in Polish law tasked with evaluating whether contracts are compliant with specific legal clauses. You will compare the assistant's outlined requirements against the user-submitted contract text. Respond with YES if the contract complies with all requirements, NO if it does not, along with an explanation of the discrepancies, or I CAN'T SAY if the text is irrelevant to the requirements. Your analysis should consider the scope, context, detailed requirements, exceptions, and any ambiguities in the contract text. Justify your response based on a detailed comparative analysis, avoid over thinking."},
            # {"role": "system", "content": conversation_history_str},
            {"role": "assistant", "content": assistant_content},
            {"role": "user", "content": user_input}
        ]
    )
    
answer = response.choices[0].message.content
print(answer)

In [56]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

client=OpenAI()

MODEL="gpt-4"
client.api_key = os.getenv("OPENAI_API_KEY")

assistant_content = '''
The contract should include an overall limitation of liability for the benefit of the Contractor with a limit not exceeding 100% of the Contract Price.

The limit should apply for all damages, liquidated damages/penalties, claims for compensation and indemnities regardless of the legal basis of the claims.
 
 Some exceptions may be accepted under exceptional circumstances:
•	 mandatory/strict product liability,
•	 mandatory third party claims for bodily injury and death,
•	 willful misconduct or fraud,

'''

user_input = '''
25.4	The Contractor’s maximum aggregate liability under this Contract, regardless of the legal theory upon which it is based, including but not limited to liability in contract or tort (including negligence and strict liability), under warranty, or otherwise, shall not exceed the Total Contract Price.

25.8	Notwithstanding anything to the contrary in this Contract, Clause 25.4 shall not operate to limit any:
(i)	liability arising from breach of laws or regulations;
(ii)	liability arising out of fraud, fraudulent misrepresentation, willful misconduct;  
(iii)	liability for death or injury, negligently caused by a Party including its Affiliates, officers, employees, agents or contractors; 
(iv)	liability for other claims which cannot be excluded or limited under mandatory applicable law.

'''
response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "You are a helpful advisor who looks for inconsistencies in contracts from clients in legal matters between large corporations, answer each question with professional language, you will answer I CAN'T SAY if user contains text which is irrelevant to assistan requirements, answer YES if the text given by the user does not contain inconsistencies with the assistant's requirements, or you will answer NO if the text given by the user contains inconsistencies with the assistant's requirements and say why"},
            # {"role": "system", "content": conversation_history_str},
            {"role": "assistant", "content": assistant_content},
            {"role": "user", "content": user_input}
        ]
    )
    
answer = response.choices[0].message.content
print(answer)

YES


In [57]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

client=OpenAI()

MODEL="gpt-4"
client.api_key = os.getenv("OPENAI_API_KEY")

assistant_content = '''
The contract should include an overall limitation of liability for the benefit of the Contractor with a limit not exceeding 100% of the Contract Price.

The limit should apply for all damages, liquidated damages/penalties, claims for compensation and indemnities regardless of the legal basis of the claims.
 
 Some exceptions may be accepted under exceptional circumstances:
•	 mandatory/strict product liability,
•	 mandatory third party claims for bodily injury and death,
•	 willful misconduct or fraud,

'''

user_input = '''
36.11 Vitiation Indemnity
The Contractor shall indemnify and hold harmless the Owner, the Lenders and their respective employees, officers, directors and agents (and the Owner acts for itself in connection with such indemnity and as trustee for the Lenders and the employees, officers, directors and agents of the Owner and the Lenders) from and against any and all suits, actions, administrative proceedings, claims, fines, losses, demands, costs, charges and expenses of whatsoever nature arising out of or resulting from any vitiating acts for which the Contractor is responsible in respect of any policy of insurance (including fraud, material misrepresentation, non-disclosure or breach of any warranty or condition of any policy by the Contractor).

51. Liability
The aggregate liability of the Contractor to the Owner under the Contract shall not exceed the Contract Price.

Except that this Clause shall not limit the liability of the Contractor:

51.2.1	in respect of loss or damage to the Works under Clause 35.2 (Responsibility for Reinstatement) to the extent available proceeds of Project insurances exceed the Contract Price; or 

51.2.2	in respect of the indemnities given in Clauses 6.7 (Laws of Poland), 30.2 (Intellectual Property Rights), 34.6 (Warranty and Indemnity as to Title to Materials and Equipment), Clause 35.4 (Injury to Persons and Property) and 36.11 (Vitiation Indemnity); or

51.2.3	in respect of termination of the Contractor’s employment in respect of the Works pursuant to Clause 42.1.5 (Notice of Termination); or

51.2.4	in cases of fraud, wilful misconduct, illegal or unlawful acts (including breach of any United State Foreign Corrupt Practices Act, United Kingdom Anti-Terrorism, Crime and Security Act 2001 or similar legislation).
[…]
The aggregate liability of the Owner to the Contractor under the Contract shall not exceed the Contract Price. Except that this Clause shall not limit the liability of the Owner in cases of fraud, wilful misconduct, illegal or unlawful acts (including breach of any United State Foreign Corrupt Practices Act, United Kingdom Anti-Terrorism, Crime and Security Act 2001 or similar legislation).
[…]
Except in respect of any liquidated damages payable under this Contract and in respect of those Clauses and other matters referred to in Clause 51.2 (Limitation of Contractor’s Aggregate Liability) and Clause 51.3 (Limitation of Owner´s Aggregate Liability) [and in respect of the indemnity given in Clause 36.1.3 (Owner’s Insurances)] [Outstanding] neither the Owner nor the Contractor shall be liable to the other by way of indemnity or by reason of any breach of the Contract or by reason of tort (including negligence or otherwise) for any loss of profit, loss of use, loss of production, loss of contracts, data, interest payments (which shall not include default interest payable pursuant to Clause 22.6 (Default in making timely payment)) or for any financial or economic loss or for any indirect or consequential loss whatsoever that may be suffered by the other.
[…]
The Owner and the Contractor intend that their respective rights, obligations and liabilities as provided for in these Conditions of Contract shall be exhaustive of the rights, obligations and liabilities of each of them to the other arising out of, under or in connection with the Contract or the Works, whether such rights, obligations and liabilities arise in respect or in consequence of an indemnity or warranty or by reason of any breach of the Contract or of statutory duty or by reason of tort (including negligence or strict liability) or otherwise. Accordingly, the remedies expressly stated in this Contract are intended to be the sole and exclusive remedies of the Parties to this Contract for the liabilities of the Parties arising out of or in connection with the Works or this Contract, notwithstanding any remedy otherwise available at law or in equity. This limitation and exclusion of liability shall apply to the extent consistent with mandatory law and regardless of legal theory and shall also apply for the benefit of employees, agents and subcontractors of such Party.

'''
response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "You are a helpful advisor who looks for inconsistencies in contracts from clients in legal matters between large corporations, answer each question with professional language, you will answer I CAN'T SAY if user contains text which is irrelevant to assistan requirements, answer YES if the text given by the user does not contain inconsistencies with the assistant's requirements, or you will answer NO if the text given by the user contains inconsistencies with the assistant's requirements and say why"},
            # {"role": "system", "content": conversation_history_str},
            {"role": "assistant", "content": assistant_content},
            {"role": "user", "content": user_input}
        ]
    )
    
answer = response.choices[0].message.content
print(answer)

NO

The provided text seems to contain an inconsistency with the terms stated in this agreement, particularly concerning the applicability of the overall limitation of liability of the Contractor. According to your original request, the Contractor's limitation of liability should apply to all damages, liquidated damages/penalties, claims for compensation and indemnities regardless of their legal basis except in certain exceptional circumstances (mandatory/strict product liability, mandatory third party claims for bodily injury and death, willful misconduct or fraud). However, in the provided contract clauses, the limitation of liability of the Contractor seems to be limited in various cases beyond the exceptions stated in your initial request, in clauses 51.2.1, 51.2.2, 51.2.3, and 51.2.4. 

The Contract Price limitation is not applicable in respect to losses or damages to the Works, indemnities in certain clauses, in cases of termination of the Contractor’s employment, and even in cas

In [41]:
import urllib.request
import json
def GPT4(system: str,assistant: str, user:str):
    url = "https://lr-lm-sandbox-ams.azure-api.net/language-model-sandbox-legal-and-regulatory-v2-prod/api/chat-completions"

    # Przygotowanie nagłówków żądania
    hdr = {
        'Content-Type': 'application/json',
        'Cache-Control': 'no-cache',
        'Ocp-Apim-Subscription-Key': '5cfae7646a2445008ca53f758b04326f',
    }

    # Dane, które chcesz wysłać
    data = {
        "user_id": "LH2024-team05@wolterskluwer.com",
        "chat_completions_config": {
            "model_name": "gpt-4",
            "model_version": "1106-Preview",
            "temperature": 0.1,
            "max_tokens": 4000,
            "top_p": 1,
            "frequency_penalty": 1,
            "presence_penalty": 1
        },
        "prompt": [
            {"role": "system","content": system},
            {"role": "assistant","content": assistant},
            {"role": "user","content": user}
        ]
    }

    # Konwersja słownika Pythona na ciąg JSON
    data = json.dumps(data).encode('utf-8')

    try:
        # Utworzenie żądania z danymi i nagłówkami
        req = urllib.request.Request(url, data=data, headers=hdr, method='POST')
        
        # Wysyłanie żądania i odbieranie odpowiedzi
        with urllib.request.urlopen(req) as response:
            response_body = response.read()
            decoded_response = response_body.decode('utf-8')
            response_dict = json.loads(decoded_response)
            
            # Wypisanie tylko części 'openai_response'
            z = response_dict.get("openai_response") # Użycie get() zapobiega błędom, jeśli klucz nie istnieje
            x=str(z)
        return x
    except Exception as e:
        print(e)
        return None

In [42]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock

insertion_lock = Lock()

def execute_task(client, MODEL, assistant_content, user_input):
    response = GPT4(
        system="You are a legal advisor with expertise in Polish law tasked with evaluating whether contracts are compliant with specific legal clauses. You will compare the assistant's outlined requirements against the user-submitted contract text. Respond with YES if the contract complies with all requirements, NO if it does not, along with an explanation of the discrepancies, or I CAN'T SAY if the text is irrelevant to the requirements. Your analysis should consider the scope, context, detailed requirements, exceptions, and any ambiguities in the contract text. Justify your response based on a detailed comparative analysis, avoid over thinking, Your response should be based on a direct, factual comparison of the clauses, within the specified legal framework.",
        assistant=assistant_content,
        user=user_input
    )
    # response = client.chat.completions.create(
    #     model=MODEL,
    #     messages=[
    #         {"role": "system", "content": "You are a legal advisor with expertise in Polish law tasked with evaluating whether contracts are compliant with specific legal clauses. You will compare the assistant's outlined requirements against the user-submitted contract text. Respond with YES if the contract complies with all requirements, NO if it does not, along with an explanation of the discrepancies, or I CAN'T SAY if the text is irrelevant to the requirements. Your analysis should consider the scope, context, detailed requirements, exceptions, and any ambiguities in the contract text. Justify your response based on a detailed comparative analysis, avoid over thinking, Your response should be based on a direct, factual comparison of the clauses, within the specified legal framework."},
    #         {"role": "assistant", "content": assistant_content},
    #         {"role": "user", "content": user_input}
    #     ]
    # )
    return response

def ten_thread_prompt(assistan: str, user: str, insertion_lock):
    ten_responses = []
    # Parameters for the task
    MODEL = "gpt-4"
    assistant_content = assistan
    user_input = user

    # Create a ThreadPoolExecutor
    with ThreadPoolExecutor(max_workers=10) as executor:
        # Submit tasks to the executor
        futures = [executor.submit(execute_task, client, MODEL, assistant_content, user_input) for _ in range(10)]

        # Process as tasks complete
        for future in as_completed(futures):
            response = future.result()
            #content = response.choices[0].message.content
            with insertion_lock:
                ten_responses.append(response)
            print(response)
    return ten_responses

In [45]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

client=OpenAI()
client.api_key = os.getenv("OPENAI_API_KEY")

assistant_content = '''
The contract should include an overall limitation of liability for the benefit of the Contractor with a limit not exceeding 100% of the Contract Price.

The limit should apply for all damages, liquidated damages/penalties, claims for compensation and indemnities regardless of the legal basis of the claims.
 
 Some exceptions may be accepted under exceptional circumstances:
•	 mandatory/strict product liability,
•	 mandatory third party claims for bodily injury and death,
•	 willful misconduct or fraud,
'''

user_input = '''
The Contractor shall indemnify and hold harmless the Employer against and from all damages, losses and/or any claims from third parties and expenses (including legal fees and expenses) resulting from breaches of this Sub-Clause. In case of any such third parties' claim and/or initiation of any administrative, misdemeanour, criminal and/or litigation procedure, the Employer shall notify the Contractor without delay and the Contractor shall without delay take all actions necessary to protect and hold harmless the Employer and/or their responsible persons. The Employer may not settle any lawsuits or matter relating to the liability of the Contractor under this Sub-Clause without the prior written consent of the Contractor. The Contractor shall have right to participate in any defence of a claim and/or to be represented by a legal professional of his own choice and his own expense.
[…]
However, this Sub-Clause shall not limit the Employer's liability under his indemnification obligations, or the Employer's liability in any case of fraud, deliberate default or reckless misconduct by the Employer.
[…]
The Employer shall indemnify and hold harmless the Contractor, the Contractor's Personnel, and their respective agents, against and from all claims, damages, losses and expenses (including legal fees and expenses) in respect of (1) bodily injury, sickness, disease or death, which is attributable to any negligence, wilful act or breach of the Contract by the Employer, the Employer's Personnel, or any of their respective agents, and (2) the matters for which liability may be excluded from insurance cover, as described in sub-paragraphs (d)(i), (ii) and (iii) of Sub-Clause 18.3 [Insurance Against Injury to Persons and Damage to Property].
[…]
The total liability of the Contractor to the Employer, under or in connection with the Contract other than under Sub-Clause 4.19 [Electricity, Water and Gas], Sub-Clause 4.20 [Employer's Equipment and Free-Issue Material], Sub-Clause 17.1 [Indemnities] and Sub-Clause 17.5 [Intellectual and Industrial Property Rights], shall not exceed the sum stated in the Particular Conditions or (if a sum is not so stated) the Contract Price stated in the Contract Agreement.

This Sub-Clause shall not limit liability in any case of fraud, deliberate default, reckless misconduct or gross negligence by the defaulting Party.

'''

response_10 = ten_thread_prompt(
    assistan=assistant_content,
    user=user_input,
    insertion_lock=insertion_lock
    )

YES

The contract text provided appears to comply with the outlined requirements. It specifies that the total liability of the Contractor is limited, as it states "The total liability of the Contractor to the Employer...shall not exceed the sum stated in the Particular Conditions or (if a sum is not so stated) the Contract Price stated in the Contract Agreement." This aligns with your requirement for an overall limitation of liability for the benefit of the Contractor with a limit not exceeding 100% of the Contract Price.

Furthermore, exceptions are noted where this limitation does not apply: "This Sub-Clause shall not limit liability in any case of fraud, deliberate default, reckless misconduct or gross negligence by the defaulting Party." These exceptions are consistent with those you've listed (willful misconduct or fraud), and while it doesn't explicitly mention mandatory/strict product liability or third-party claims for bodily injury and death, another part of your provided text

In [48]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

client=OpenAI()
client.api_key = os.getenv("OPENAI_API_KEY")

system = "You are a legal advisor with expertise in Polish law tasked with evaluating whether contracts are compliant with specific legal clauses, You will compare ten answers from user input and create one that is coherent in professional language, based on assistant requirement, you will answer I CAN'T SAY if the provided texts from user are irrelevant to the assistant's requirements, you will answer YES if the texts provided from user does not contain any inconsistencies with the assistant's requirements, or you will answer NO if the texts provided from user contains inconsistencies with the requirements assistant and tell me why. Justify your response based on a detailed comparative analysis, avoid over thinking, Your response should be based on a direct, factual comparison of the clauses, within the specified legal framework."
assistant_content = '''
The contract should include an overall limitation of liability for the benefit of the Contractor with a limit not exceeding 100% of the Contract Price.

The limit should apply for all damages, liquidated damages/penalties, claims for compensation and indemnities regardless of the legal basis of the claims.
 
 Some exceptions may be accepted under exceptional circumstances:
•	 mandatory/strict product liability,
•	 mandatory third party claims for bodily injury and death,
•	 willful misconduct or fraud,
'''
user = "{},{},{},{},{},{},{},{},{},{}".format(*response_10)

response = GPT4(
    system=system,
    assistant=assistant_content,
    user=user
)
    
#answer = response.choices[0].message.content
print(response)

I CAN'T SAY
